### Семинар 3. N-gram language models

Обучать языковую модель будем на открытом корпусе русского языка.<br/>
Работать с ним удобно с помощью модуля opencorpora-tools, который предоставляет NLTK-подобный интерфейс к нему.

In [1]:
# Установим opencorpora-tools
# Если у вас anaconda, лучше используйте команду conda вместо pip.
!pip install opencorpora-tools

In [2]:
# Скачаем корпус
!opencorpora download

Traceback (most recent call last):
  File "/Users/hope/.pyenv/versions/py346/bin/opencorpora", line 3, in <module>
    from opencorpora import cli
  File "/Users/hope/.pyenv/versions/3.4.6/envs/py346/lib/python3.4/site-packages/opencorpora/__init__.py", line 4, in <module>
    from .reader_lxml import load
  File "/Users/hope/.pyenv/versions/3.4.6/envs/py346/lib/python3.4/site-packages/opencorpora/reader_lxml.py", line 8, in <module>
    from lxml import etree
ImportError: dlopen(/Users/hope/.pyenv/versions/3.4.6/envs/py346/lib/python3.4/site-packages/lxml/etree.so, 2): Library not loaded: libxml2.2.dylib
  Referenced from: /Users/hope/.pyenv/versions/3.4.6/envs/py346/lib/python3.4/site-packages/lxml/etree.so
  Reason: Incompatible library version: etree.so requires version 12.0.0 or later, but libxml2.2.dylib provides version 10.0.0


In [4]:
!ls # Файл с корпусом должен был появиться в текущей папке

3. Языковые модели на N-граммах.ipynb
annot.opcorpora.xml
annot.opcorpora.xml.~
Летучка.txt


In [2]:
import opencorpora
corpus = opencorpora.CorpusReader('annot.opcorpora.xml')
docs = corpus.catalog()

In [3]:
docs[:10]

[('1', '"Частный корреспондент"'),
 ('2', '00021 Школа злословия'),
 ('3', '00022 Последнее восстание в Сеуле'),
 ('4', '00023 За кота - ответишь!'),
 ('5', '00024 Быстротечный кинороман'),
 ('6', '00014 Холодная ванна возвращает силы'),
 ('7', '00031 Рецессия в Латвии и Эстонии'),
 ('8', 'Википедия'),
 ('9', '06037 100 дней Обамы: iТоги'),
 ('10', '01961 100 миллиардов может и не хватить')]

In [4]:
# Выведем несколько предложений из корпуса.
for id, name in docs[:2]:
    # Можно использовать corpus.tagged_sents(id), чтобы отсеять пунктуаторы.
    sents = corpus.sents(id)
    for words in sents[:5]:
        
        print(words)

['«', 'Школа', 'злословия', '»', 'учит', 'прикусить', 'язык']
['Сохранится', 'ли', 'градус', 'дискуссии', 'в', 'новом', 'сезоне', '?']
['Великолепная', '«', 'Школа', 'злословия', '»', 'вернулась', 'в', 'эфир', 'после', 'летних', 'каникул', 'в', 'новом', 'формате', '.']
['В', 'истории', 'программы', 'это', 'уже', 'не', 'первый', '«', 'ребрендинг', '»', '.']
['Сейчас', 'с', 'трудом', 'можно', 'припомнить', ',', 'что', 'начиналась', '«', 'Школа', '…', '»', 'на', 'канале', '«', 'Культура', '»', 'как', 'стандартное', 'ток-шоу', ',', 'которое', 'отличалось', 'от', 'других', '«', 'кухонными', '»', 'обсуждениями', 'гостя', ',', 'что', 'называется', '–', '«', 'за', 'глаза', '»', ',', 'и', 'неожиданными', 'персонами', 'в', 'качестве', 'ведущих', '.']


In [5]:
def train_model(docs, n):
    ngramms = {}
    END_OF_SENTENCE = "<END>"
    for id, name in docs:
        sents = corpus.sents(id)
        for words in sents:
            words.append(END_OF_SENTENCE)
            
            for i in range(n, len(words)):
                ngramma = "_".join(words[i - (n-1):i])
                if ngramms.get(ngramma) == None:
                    ngramms[ngramma] = {}
                if ngramms[ngramma].get(words[i]) == None:
                    ngramms[ngramma][words[i]] = 0
                ngramms[ngramma][words[i]] += 1
    return ngramms
                

Напишите функцию train_model(docs, n), которая возвращает обученную n-граммную модель

Модель предлагается сделать в виде отображения: <br/>
(n-1)-грамма => {следующее слово => вероятность всей n-грамы}<br/>
или<br/>
(n-1)-грамма => {следующее слово => количество соответствующих n-грамм в корпусе}

Введите специальное слово, обозначающее конец предложения.<br/>Добавляйте его в n-граммы, которые находятся в конце предложения

In [6]:
from collections import defaultdict

def train_model(docs, n):
    # model = {(word_1, ..., word_n-1): {word_n: 0,2, word_n: 0,1}}
    model = defaultdict(lambda: defaultdict(int))
    
    END_OF_SENTENCE = "<END>"
    for id, name in docs:
        sents = corpus.sents(id)
        for words in sents:
            words.append(END_OF_SENTENCE)
            
            for i in range(n, len(words)):
                ngramma = "_".join(words[i - (n-1):i])
                model[ngramma][words[i]] += 1
    return model

In [7]:
model = train_model(docs, 3)

Напишите функцию get_sentence_probability(model, tokens), которая будет возвращать вероятность предложения в n-граммной модели

In [12]:
def get_sentence_probability(model, tokens):
    n = 1
    keys = list(model)
    if len(keys) > 1:
        n += 1 + keys[0].count("_")
    if len(tokens) < n:
        return 0.0
    prob = 1.0
    for i in range(n-1, len(tokens)):
        ngramma = "_".join(words[i - (n-1):i])
        sNGramm = sum(model[ngramma].values())
        curProb = 0
        if sNGramm != 0:
            curProb = model[ngramma][words[i]]/sNGramm
        prob *= curProb
    return prob
        

In [170]:
get_sentence_probability(model, "Статья 5. Независимость судей арбитражных судов".split())

2.976540128682568e-22

Напишите функцию generate_sentence(model, max_sentence_length), которая будет генерировать случайные предложения из распределения, описываемого моделью model.

In [160]:
import random

def generate_next_word(model, prevNWords):  
    #print(model[prevNWords])
    nextPossibleWords = list(model[prevNWords].keys())
    probs = list(model[prevNWords].values())
    probsCount = len(nextPossibleWords)
    
    choice = random.uniform(0.0, sum(probs))
    curSum = 0.0
    for i in range(probsCount):
        curSum += probs[i]
        if choice <= curSum:
            return nextPossibleWords[i]

def generate_sentence(model, max_sentence_length):
    n = 1
    keys = list(model)
    if len(keys) > 1:
        n += 1 + keys[0].count("_")
    
    startWord = keys[random.randint(0, len(keys)-1)]
    sent = startWord.split("_")
    prevNWord = startWord

    for i in range(n-1, max_sentence_length):

        nextWord = generate_next_word(model, prevNWord)
        if nextWord == "<END>":
            break

        sent.append(nextWord)
        if n != 1:
            prevNWord = "_".join(sent[-n+1:])

    return " ".join(sent)   
        
    

Посмотрите, какие предложения генерирует unigram-модель, bigram-модель, trigram-модель

In [161]:
model1 = train_model(docs, 1)

In [162]:
model2 = train_model(docs, 2)

In [163]:
model3 = train_model(docs, 3)

In [165]:
print(generate_sentence(model1, 7))
print(generate_sentence(model2, 7))
print(generate_sentence(model3, 7))

 разделением сослагательном оформит пополнении сдать мероприятие (
санатория , а не берет книгу Нормана
ситуацию решает он , что вопросы обеспечения


Напишите функцию suggest, которая будет предсказывать следующее слово для слов tokens

In [173]:
def suggest(model, tokens):
    n = 1
    keys = list(model)
    if len(keys) > 1:
        n += 1 + keys[0].count("_")

    lastNTokens = "_".join(tokens[-n+1:])
    countToToken = {value:key for key, value in model[lastNTokens].items()}
    maxCount = sorted(list(countToToken), reverse=True)[0]
    return countToToken[maxCount]
    

In [176]:
suggest(model3, "Сохранится ли градус".split( " "))

'дискуссии'

Перепишите функцию train_model так, чтобы она использовала сглаживание вероятностей n-грамм

In [114]:
def train_model(docs, n):
    # model = {(word_1, ..., word_n-1): {word_n: 0,2, word_n: 0,1}}
    model = defaultdict(lambda: defaultdict(lambda: 1))
    
    END_OF_SENTENCE = "<END>"
    for id, name in docs:
        sents = corpus.sents(id)
        for words in sents:
            words.append(END_OF_SENTENCE)
            
            for i in range(n, len(words)):
                ngramma = "_".join(words[i - (n-1):i])
                model[ngramma][words[i]] += 1
    return model